In [33]:
import numpy as np
import pandas as pd
# from icecream import ic
import subprocess
import json
import os
import plotly.graph_objects as go
from matplotlib import pyplot as plt
# import geopandas as gpd
# from shapely.geometry import Polygon
# from shapely.ops import cascaded_union, unary_union
# from shapely import geometry
# from geopy import distance
%matplotlib inline
# import plotly.express as px 
from scipy import signal
from tqdm import tqdm
# from Bio import Phylo
# import ete3
import itertools
import sklearn

In [2]:
df = pd.read_csv('data1/CRyPTIC_reuse_table_20221019_cleaned.csv')


In [13]:
df.columns

Index(['UNIQUEID', 'AMI_BINARY_PHENOTYPE', 'BDQ_BINARY_PHENOTYPE',
       'CFZ_BINARY_PHENOTYPE', 'DLM_BINARY_PHENOTYPE', 'EMB_BINARY_PHENOTYPE',
       'ETH_BINARY_PHENOTYPE', 'INH_BINARY_PHENOTYPE', 'KAN_BINARY_PHENOTYPE',
       'LEV_BINARY_PHENOTYPE', 'LZD_BINARY_PHENOTYPE', 'MXF_BINARY_PHENOTYPE',
       'RIF_BINARY_PHENOTYPE', 'RFB_BINARY_PHENOTYPE', 'AMI_MIC', 'BDQ_MIC',
       'CFZ_MIC', 'DLM_MIC', 'EMB_MIC', 'ETH_MIC', 'INH_MIC', 'KAN_MIC',
       'LEV_MIC', 'LZD_MIC', 'MXF_MIC', 'RIF_MIC', 'RFB_MIC',
       'AMI_PHENOTYPE_QUALITY', 'BDQ_PHENOTYPE_QUALITY',
       'CFZ_PHENOTYPE_QUALITY', 'EMB_PHENOTYPE_QUALITY',
       'ETH_PHENOTYPE_QUALITY', 'INH_PHENOTYPE_QUALITY',
       'KAN_PHENOTYPE_QUALITY', 'LEV_PHENOTYPE_QUALITY',
       'LZD_PHENOTYPE_QUALITY', 'MXF_PHENOTYPE_QUALITY',
       'RIF_PHENOTYPE_QUALITY', 'RFB_PHENOTYPE_QUALITY', 'ENA_SAMPLE', 'VCF',
       'REGENOTYPED_VCF', 'wgs_id'],
      dtype='object')

In [41]:
df_pheno = df[['AMI_BINARY_PHENOTYPE', 'BDQ_BINARY_PHENOTYPE',
       'CFZ_BINARY_PHENOTYPE', 'DLM_BINARY_PHENOTYPE', 'EMB_BINARY_PHENOTYPE',
       'ETH_BINARY_PHENOTYPE', 'INH_BINARY_PHENOTYPE', 'KAN_BINARY_PHENOTYPE',
       'LEV_BINARY_PHENOTYPE', 'LZD_BINARY_PHENOTYPE', 'MXF_BINARY_PHENOTYPE',
       'RIF_BINARY_PHENOTYPE', 'RFB_BINARY_PHENOTYPE']]
# Correcting the approach to count 'S', 'I', and 'R' for each column and save in a new dataframe

# Importing pandas for dataframe manipulation
import pandas as pd

# Assuming df is the initial dataframe
# Creating a new DataFrame to store counts
counts_df = pd.DataFrame(columns=['S', 'I', 'R'])

# Looping through each column in the original dataframe
for column in df_pheno.columns:
    # Counting occurrences of 'S', 'I', and 'R'
    s_count = (df[column] == 'S').sum()
    i_count = (df[column] == 'I').sum()
    r_count = (df[column] == 'R').sum()
    
    # Adding the counts to the new dataframe
    counts_df.loc[column] = [s_count, i_count, r_count]

counts_df['Phenotype'] = counts_df.index.str.replace('_BINARY_PHENOTYPE', '')


In [59]:
counts_df

,S,I,R,Phenotype,Total,S%,I%,R%
AMI_BINARY_PHENOTYPE,11186,0,882,AMI,12068,92.691415,0.0,7.308585
BDQ_BINARY_PHENOTYPE,11955,0,109,BDQ,12064,99.096485,0.0,0.903515
CFZ_BINARY_PHENOTYPE,11520,0,525,CFZ,12045,95.641345,0.0,4.358655
DLM_BINARY_PHENOTYPE,11737,0,186,DLM,11923,98.43999,0.0,1.56001
EMB_BINARY_PHENOTYPE,8334,1559,2261,EMB,12154,68.570018,12.827053,18.602929
ETH_BINARY_PHENOTYPE,9429,972,1727,ETH,12128,77.745712,8.014512,14.239776
INH_BINARY_PHENOTYPE,6159,0,5907,INH,12066,51.044257,0.0,48.955743
KAN_BINARY_PHENOTYPE,11006,0,1120,KAN,12126,90.763648,0.0,9.236352
LEV_BINARY_PHENOTYPE,10014,0,2145,LEV,12159,82.358747,0.0,17.641253
LZD_BINARY_PHENOTYPE,12029,0,156,LZD,12185,98.719737,0.0,1.280263


In [58]:
colors = ['#28A745', '#E0ACD5', '#DC3545']  # Light blue, very light blue, light grey

counts_df['Total'] = counts_df['S'] + counts_df['I'] + counts_df['R']

counts_df['S%'] = (counts_df['S'] / counts_df['Total']) * 100
counts_df['I%'] = (counts_df['I'] / counts_df['Total']) * 100
counts_df['R%'] = (counts_df['R'] / counts_df['Total']) * 100

fig = go.Figure(data=[
    go.Bar(name='S', x=counts_df["Phenotype"], y=counts_df["S"], marker_color=colors[0]),
    go.Bar(name='I', x=counts_df["Phenotype"], y=counts_df["I"], marker_color=colors[1]),
    go.Bar(name='R', x=counts_df["Phenotype"], y=counts_df["R"], marker_color=colors[2])
])

# Change the bar mode
fig.update_layout(barmode='stack', title='Stacked Bar Chart of Phenotypes',
                  xaxis_title="", yaxis_title="%",
                  xaxis={'categoryorder':'total descending'}, template="simple_white",
                  width=600,  # Adjust the width to make the plot more narrow
                  height=600,)

fig.update_traces(width=0.7)

# Show figure
fig.show()

In [9]:
df[['EMB_BINARY_PHENOTYPE']].value_counts()

EMB_BINARY_PHENOTYPE
S                       8334
R                       2261
I                       1559
dtype: int64

In [6]:
df[['AMI_BINARY_PHENOTYPE', 'BDQ_BINARY_PHENOTYPE',
       'CFZ_BINARY_PHENOTYPE', 'DLM_BINARY_PHENOTYPE', 'EMB_BINARY_PHENOTYPE',
       'ETH_BINARY_PHENOTYPE', 'INH_BINARY_PHENOTYPE', 'KAN_BINARY_PHENOTYPE',
       'LEV_BINARY_PHENOTYPE', 'LZD_BINARY_PHENOTYPE', 'MXF_BINARY_PHENOTYPE',
       'RIF_BINARY_PHENOTYPE', 'RFB_BINARY_PHENOTYPE']].value_counts()

AMI_BINARY_PHENOTYPE  BDQ_BINARY_PHENOTYPE  CFZ_BINARY_PHENOTYPE  DLM_BINARY_PHENOTYPE  EMB_BINARY_PHENOTYPE  ETH_BINARY_PHENOTYPE  INH_BINARY_PHENOTYPE  KAN_BINARY_PHENOTYPE  LEV_BINARY_PHENOTYPE  LZD_BINARY_PHENOTYPE  MXF_BINARY_PHENOTYPE  RIF_BINARY_PHENOTYPE  RFB_BINARY_PHENOTYPE
S                     S                     S                     S                     S                     S                     S                     S                     S                     S                     S                     S                     S                       4608
                                                                                                                                    R                     S                     S                     S                     S                     S                     S                        647
                                                                                        I                     S                  